In [1]:
import pandas as pd
import gtfs_kit as gk
import helper #homemade function file helper.py
import warnings
import numpy as np
import seaborn as sns
import dataframe_image as dfi
import json
import os
from datetime import datetime

warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [2]:
lista_fermate = ['PISA CENTRALE','PISA SAN ROSSORE','S.GIULIANO TERME','RIGOLI','RIPAFRATTA','LUCCA','S. PIETRO A VICO',
                 'PONTE A MORIANO','DIECIMO - PESCAGLIA','BORGO A MOZZANO','BAGNI DI LUCCA','GHIVIZZANO - COREGLIA','FORNACI DI BARGA',
                 'BARGA GALLICANO','CASTELVECCHIO PASCOLI','FOSCIANDORA - CESERANA','CASTELNUOVO DI GARFAGNANA','VILLETTA S. ROMANO','POGGIO - CAREGGINE - VAGLI',
                 'CAMPORGIANO','PIAZZA AL SERCHIO','MINUCCIANO - PIEVE - CASOLA','EQUI TERME','MONZONE-M.DEI BANCHI-ISOLANO','GRAGNOLA',
                 'FIVIZZANO - GASSANO','FIVIZZANO ROMETTA - SOLIERA','AULLA LUNIGIANA']

mappa_location_codice = {
              'PISA CENTRALE':'S06500_1',
              'PISA SAN ROSSORE':'S06501_1',
              'S.GIULIANO TERME':'S06400_1',
              'RIGOLI':'S06401_1',
              'RIPAFRATTA':'S06402_1',
              'LUCCA':'S06404_1',
              'S. PIETRO A VICO':'S06313_1',
              'PONTE A MORIANO':'S06312_1',
              'DIECIMO - PESCAGLIA':'S06310_1',
              'BORGO A MOZZANO':'S06309_1',
              'BAGNI DI LUCCA':'S06308_1',
              'GHIVIZZANO - COREGLIA':'S06306_1',
              'FORNACI DI BARGA':'S06304_1',
              'BARGA GALLICANO':'S06303_1',
              'CASTELVECCHIO PASCOLI':'S06302_1',
              'FOSCIANDORA - CESERANA':'S06301_1',
              'CASTELNUOVO DI GARFAGNANA':'S06300_1',
              'VILLETTA S. ROMANO':'S06322_1',
              'POGGIO - CAREGGINE - VAGLI':'S06323_1',
              'CAMPORGIANO':'S06324_1',
              'PIAZZA AL SERCHIO':'S06325_1',
              'MINUCCIANO - PIEVE - CASOLA':'S06227_1',
              'EQUI TERME':'S06226_1',
              'MONZONE-M.DEI BANCHI-ISOLANO':  'S06225_1',
              'GRAGNOLA':'S06224_1',
              'FIVIZZANO - GASSANO':'S06223_1',
              'FIVIZZANO ROMETTA - SOLIERA':'S06222_1',
              'AULLA LUNIGIANA':'S06013_1'}

directory = '../../realtime_data'

In [3]:
delays_df =  pd.DataFrame(columns=['n','p','rp','a','ra','c','oaz','opz','op','oa','fr','pr','date'])

for filename in os.listdir(directory):
    file = os.path.join(directory, filename)
    if os.path.isfile(file):
        #df = pd.read_json('../../realtime_data/dati_01_01_2023.json')
        f = open(file)
        
        #Calcolo data in base al Nome del File
        data = json.load(f)
        giorno = f.name[25:35].split('_')
        date_to_add = giorno[2] + giorno[1] + giorno[0]
        
        #Load file in Dataframe
        df = pd.DataFrame(data['treni'])
        # only interested route
        df = df[df.p.isin(lista_fermate) & df.a.isin(lista_fermate)]
        df = df.reset_index(drop=True)
        ## Add date columns
        df['date'] = date_to_add
        ## Drop columns with all NAN
        df.dropna(axis=1, how='all')
        #Create one big dataset for delays
        delays_df = pd.concat([delays_df, df]).reset_index(drop=True)


In [4]:
delays_df = delays_df.rename(columns={'n': 'codice', 'p': 'punto_partenza','rp':'ritardo_partenza','a':'punto_arrivo',
                                     'ra':'ritardo_arrivo','c':'categoria'})
delays_df.dropna(axis=1, how='all')


,codice,punto_partenza,ritardo_partenza,punto_arrivo,ritardo_arrivo,categoria,oaz,opz,op,oa,fr,pr,date,dl,oo,od,cn
0,19192,PIAZZA AL SERCHIO,2,AULLA LUNIGIANA,2,REG,1662349440,1662347100,1662347100,1662349440,"[{'n': 'PIAZZA AL SERCHIO', 'ra': 'N', 'rp': '...",NaN,20220905,NaN,NaN,NaN,NaN
1,19194,PIAZZA AL SERCHIO,1,AULLA LUNIGIANA,-3,REG,1662352320,1662349560,1662349560,1662352320,"[{'n': 'PIAZZA AL SERCHIO', 'ra': 'N', 'rp': '...",NaN,20220905,NaN,NaN,NaN,NaN
2,18551,PISA CENTRALE,7,LUCCA,5,REG,1662352800,1662351180,1662351180,1662352800,"[{'n': 'PISA CENTRALE', 'ra': 'N', 'rp': '7', ...",NaN,20220905,NaN,NaN,NaN,NaN
3,19191,PIAZZA AL SERCHIO,4,LUCCA,-4,REG,1662352860,1662348300,1662348300,1662352860,"[{'n': 'PIAZZA AL SERCHIO', 'ra': 'N', 'rp': '...",NaN,20220905,NaN,NaN,NaN,NaN
4,18552,LUCCA,2,PISA CENTRALE,-5,REG,1662356640,1662354480,1662354480,1662356640,"[{'n': 'LUCCA', 'ra': 'N', 'rp': '2', 'oa': 0,...",NaN,20220905,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36054,19183,CASTELNUOVO DI GARFAGNANA,1,PISA CENTRALE,21,REG,1698698580,1698693300,1698693300,1698698580,"[{'n': 'CASTELNUOVO DI GARFAGNANA', 'ra': 'N',...",NaN,20231030,NaN,NaN,NaN,NaN
36055,6242,LUCCA,2,CASTELNUOVO DI GARFAGNANA,28,REG,1698699360,1698695580,1698695580,1698699360,"[{'n': 'LUCCA', 'ra': 'N', 'rp': '2', 'oa': 0,...",NaN,20231030,NaN,NaN,NaN,NaN
36056,18603,PISA CENTRALE,1,LUCCA,-3,REG,1698699120,1698697200,1698697200,1698699120,"[{'n': 'PISA CENTRALE', 'ra': 'N', 'rp': '1', ...",NaN,20231030,NaN,NaN,NaN,NaN
36057,18605,PISA CENTRALE,2,LUCCA,10,REG,1698700620,1698699000,1698699000,1698700620,"[{'n': 'PISA CENTRALE', 'ra': 'N', 'rp': '2', ...",NaN,20231030,NaN,NaN,NaN,NaN


In [5]:
delays_df

,codice,punto_partenza,ritardo_partenza,punto_arrivo,ritardo_arrivo,categoria,oaz,opz,op,oa,...,date,sub,dl,oo,od,cn,sep,ope,sea,oae
0,19192,PIAZZA AL SERCHIO,2,AULLA LUNIGIANA,2,REG,1662349440,1662347100,1662347100,1662349440,...,20220905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,19194,PIAZZA AL SERCHIO,1,AULLA LUNIGIANA,-3,REG,1662352320,1662349560,1662349560,1662352320,...,20220905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,18551,PISA CENTRALE,7,LUCCA,5,REG,1662352800,1662351180,1662351180,1662352800,...,20220905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,19191,PIAZZA AL SERCHIO,4,LUCCA,-4,REG,1662352860,1662348300,1662348300,1662352860,...,20220905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18552,LUCCA,2,PISA CENTRALE,-5,REG,1662356640,1662354480,1662354480,1662356640,...,20220905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36054,19183,CASTELNUOVO DI GARFAGNANA,1,PISA CENTRALE,21,REG,1698698580,1698693300,1698693300,1698698580,...,20231030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36055,6242,LUCCA,2,CASTELNUOVO DI GARFAGNANA,28,REG,1698699360,1698695580,1698695580,1698699360,...,20231030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36056,18603,PISA CENTRALE,1,LUCCA,-3,REG,1698699120,1698697200,1698697200,1698699120,...,20231030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36057,18605,PISA CENTRALE,2,LUCCA,10,REG,1698700620,1698699000,1698699000,1698700620,...,20231030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
delays_df.to_csv('../processed_files/delays_preprocessing_general_dataset.csv')


In [ ]:
# We now expand the dataset by having trip_id, date, station
delays_expanded = pd.DataFrame(columns=['codice','data_giorno','codice_stazione','opt','oat'])
for index, row  in delays_df.iterrows():
    if(index % 1000 == 0):
        print("row " + str(index) + "/" +str(delays_df.index.max()))
    json_to_put_in_table = str(delays_df.loc[index, 'fr']).replace("'","\"")
    if(json_to_put_in_table != "nan"):
        temp_dictionary = json.loads(json_to_put_in_table)
        df = pd.DataFrame(temp_dictionary)
        df['codice'] = row['codice']
        df['data_giorno'] = row['date']
        df.oa = df.oa.apply(lambda x: datetime.fromtimestamp(x).time())
        df.op = df.op.apply(lambda x: datetime.fromtimestamp(x).time())
        df['opt'] = row['op']
        df['oat'] = row['oa']
        delays_expanded = pd.concat([delays_expanded, df]).reset_index(drop=True)
    else:
        
        #row_partenza = {'codice': row['codice'], 'data_giorno':row['date'],'codice_stazione':'', 'n':row['punto_partenza'], 
        #                'ra': 'N', 'rp' : row['ritardo_partenza'],'oa': 0,'op': datetime.fromtimestamp(row['op']).time() }
        
        #row_arrivo = {'codice': row['codice'], 'data_giorno':row['date'], 'codice_stazione':'','n':row['punto_arrivo'], 
        #                'ra': row['ritardo_arrivo'] , 'rp' : 'N','oa': datetime.fromtimestamp(row['oa']).time(),'op': 0 }
        
        row_partenza = {'codice': row['codice'], 'data_giorno':row['date'],'codice_stazione':'', 'n':row['punto_partenza'], 
                        'ra': 'N', 'rp' : row['ritardo_partenza']}
        
        row_arrivo = {'codice': row['codice'], 'data_giorno':row['date'], 'codice_stazione':'','n':row['punto_arrivo'], 
                        'ra': row['ritardo_arrivo'] , 'rp' : 'N' }
        
        delays_expanded = pd.concat([delays_expanded, pd.DataFrame([row_partenza])], ignore_index=True)
        delays_expanded = pd.concat([delays_expanded, pd.DataFrame([row_arrivo])], ignore_index=True)

row 0/36058
row 1000/36058
row 2000/36058
row 3000/36058
row 4000/36058
row 5000/36058
row 6000/36058
row 7000/36058
row 8000/36058
row 9000/36058
row 10000/36058
row 11000/36058
row 12000/36058
row 13000/36058
row 14000/36058
row 15000/36058
row 16000/36058
row 17000/36058
row 18000/36058
row 19000/36058
row 20000/36058
row 21000/36058
row 22000/36058
row 23000/36058


In [ ]:
delays_expanded = delays_expanded.rename(columns={'n': 'stazione', 'ra': 'ritardo_arrivo',
                                                  'rp':'ritardo_partenza','oa':'orario_arrivo',
                                                  'op':'orario_partenza'})

delays_expanded["codice_stazione"] = delays_expanded["stazione"].apply(lambda x: mappa_location_codice.get(x))


In [ ]:
delays_expanded

In [ ]:
delays_expanded.to_csv('../processed_files/delays_preprocessing_expanded.csv')